# Spamlet  entropy
## PHYS 481 Assignment 2

This assignment examines the probability that random sources can generate meaningful outcomes.  Specifically, we consider how long it might take for a large number of monkeys with typewriters to eventually produce Hamlet.

Several cases are considered
1. independent distribution of letters assumed to be equally probable
2. independent distribution of letters estimated from text
2a. higher order distribution of multiple letters ie. pairs, triplets
3. joint distribution of letters ie. aa, ab, ac .. zz
4. independent distribution of words estimated from text
5. joint distribution of words

Probabilities and entropy are determined for each case.  The text of Hamlet from `gutenberg.org` is used to determine empirical frequencies.

These tasks often involve calculations that often over/underflow even with double precision floating point variables.  This can be dealt with by using logarithms.

Finally, random sequences were generated according to statistics for (in)dependent characters and words.

In [1]:
# load modules for math and plotting
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
import string

### reading data

We want to avoid pulling the file contents over the net every time that we run this cell.  This reduces uneccessary network traffic.  Perhaps more importantly, it means that we should always have a local copy of the data.  This will allow us to work even if we don't have network access, or if the original file is moved, changed, or deleted.

Note: `urllib` will return byte data, so use the `decode` method to convert to a string.

In [2]:
# Don't pull the same file every time!
#

local_filename = './phys481_hamlet.txt'
remote_url = r'http://www.gutenberg.org/files/1524/1524-0.txt'

# Could just get a local copy with browser or "wget" command.
if not os.path.exists(local_filename):
    byte_data = urllib.request.urlopen( remote_url ).read()    
    with open(local_filename,'wb') as fid:
        fid.write(byte_data)
        
with open(local_filename,'rb') as fid:
    byte_data = fid.read()
    
print( 'byte_data length:',len(byte_data)) 
char_data = byte_data.decode() #'utf-8')
print( 'char_data length:',len(char_data)) 

byte_data length: 204885
char_data length: 202379


### cleaning data

The original file is formatted with line breaks for easy reading.  For this analysis we would like a simple sequence of characters without any format codes such as newline (`\n`).  We do this by removing all non-ascii characters, and then converting everything to lower case.  Sequences with multple spaces are collapsed to single spaces.  This will allow us to obtain words by simply splitting on whitespace.

Reality checks
* There are 27 unique characters (26 letters plus space) 
* the contents of the word list seem reasonable

In [3]:
# collapse all whitespace (including newline)
# to single spaces
data = ' '.join(char_data.split())

# eliminate all non-ascii characters (except space)
# and convert to lower case
valid_chars = string.ascii_letters + ' '
char_list = [c.lower() for c in data if c in valid_chars]
print( '# of characters: ', len(char_list) )
print( 'unique characters: ', len(np.unique(char_list)) )
print( 'reality check- excerpt from char_list: ', char_list[0:999:19] )

# get words by joining with empty string and then splitting
word_list = ''.join(char_list).split()
print( '\n# of words: ', len(word_list) )
print( 'unique words: ', len(np.unique(word_list)) )
print( 'reality check- excerpt from word_list: ',  word_list[0:999:19])


# of characters:  185072
unique characters:  27
reality check- excerpt from char_list:  ['p', 'b', 'i', 't', 'h', 'y', 'd', 't', 'o', ' ', 't', 'r', 'i', 'e', 's', 't', 'l', 'o', 'g', ' ', ' ', 'y', ' ', 'u', ' ', 'n', ' ', 'i', 'e', ' ', 'd', 'g', 'r', ' ', 'e', ' ', ' ', 'a', 'd', ' ', 'i', 't', ' ', 'm', ' ', 'a', 't', 'i', 's', 'h', ' ', ' ', 'i']

# of words:  35022
unique words:  5224
reality check- excerpt from word_list:  ['project', 'united', 'may', 'this', 'the', 'date', 'this', 'i', 'scene', 'castle', 'scene', 'the', 'ii', 'scene', 'in', 'prince', 'queen', 'friend', 'francisco', 'captain', 'platform', 'unfold', 'barnardo', 'and', 'good', 'horatio', 'marcellus', 'you', 'horatio', 'barnardo', 'of', 'watch', 'to', 'ears', 'and', 'from', 'the', 'the', 'it', 'it', 'together', 'i', 'thee', 'pale', 'this', 'horatio', 'so', 'marcellus', 'horatio', 'this', 'why', 'daily', 'task']


### Factoring out repetition

For the intial version of this report I wrote some code to build a catalog of single letters, then more code to work with pairs of letters, and yet more code to work with words and pairs of words.  All of this was essentially almost the same, so I pulled out the common operations into a functions.

In [4]:
def build_catalog_single( data, group=1 ):
    ''' For a sequence (eg. list or string) of items,
    step through and assemble a dict containing the
    number of occurances of each item.
    
    group: default 1 to deal with individual items,
    higher values to deal with groups of items ie.
        data='abcd' , nitem=2
        items='ab', 'bc', 'cd'
    '''
    
    result = {}
    for indx in range(len(data)-group):
        item = ','.join( data[indx:indx+group] )
        
        if item not in result:
            result[item] = 0
        
        result[item] += 1
    
    return result

### Single letter catalog

The most commonly occuring symbol is a space, followed by an `e` (as expected).

In [5]:
char_catalog_len1 = build_catalog_single( char_list, group=1)
char_unique = sorted( char_catalog_len1.keys() )
print([(c,char_catalog_len1[c]) for c in char_unique])

[(' ', 35063), ('a', 11324), ('b', 2110), ('c', 3410), ('d', 5675), ('e', 17565), ('f', 3124), ('g', 2849), ('h', 9245), ('i', 10005), ('j', 200), ('k', 1415), ('l', 6862), ('m', 4644), ('n', 9739), ('o', 12856), ('p', 2461), ('q', 230), ('r', 9139), ('s', 9448), ('t', 14065), ('u', 4983), ('v', 1347), ('w', 3425), ('x', 206), ('y', 3554), ('z', 127)]


## Letter pair catalog

With 27 characters the largest number of posible pairs is 729.  The actual observed number is 505.  This is not surprising, as there are many combinations that are unlikely in English ie. 'zx', 'qq'.

Looking at the most likely pairs, many of them appear to come from "the" ie. ' t', 'th', 'he', 'e '

In [6]:
char_catalog_len2 = build_catalog_single( char_list, group=2)
char_unique = np.array( sorted( char_catalog_len2.keys() ) )
print('number of unique pairs: ', len(char_unique), ' max=27^2=',27*27)

# sort from smallest to largest frequency
argsort = np.argsort( [char_catalog_len2[c] for c in char_unique])
print([(c,char_catalog_len2[c]) for c in char_unique[argsort[-9:]]])

number of unique pairs:  505  max=27^2= 729
[(' ,h', 2628), ('h,e', 3208), ('d, ', 3256), (' ,a', 3282), ('s, ', 4213), ('t,h', 4351), ('t, ', 4506), (' ,t', 4830), ('e, ', 6044)]


### Single word catalog

As expected, the most commonly occuring word is `the`.

In [7]:
word_catalog_len1 = build_catalog_single( word_list, group=1)
word_unique = np.array( sorted( word_catalog_len1.keys() ) )

# sort from smallest to largest frequency
argsort = np.argsort( [word_catalog_len1[c] for c in word_unique] )
print([(c,word_catalog_len1[c]) for c in word_unique[argsort[-9:]]])
#print([(c,word_catalog_len1[c]) for c in word_unique[0:999:19]])

#plt.hist( [word_catalog_len1[c] for c in word_unique], bins=[1,2,3,4,5,6,7,8,9] )

[('in', 510), ('my', 516), ('i', 570), ('a', 611), ('you', 625), ('of', 800), ('to', 818), ('and', 1052), ('the', 1301)]


### Entropy of sequences

A random sequence drawn independently from a set of 27 symbols should have an entropy of about 4.755 bits per symbol

The spamlet text has an entropy of 4.10 bits per character, which means that spamlet is more predictable than pure randomness.
Groups of two characters convey 7.46 bits per pair, which is roughly 0.5 bits less than the 8.2 bits expected from independent characters.  The entropy increases by smaller amounts with each additional character, so that 5 character groups have only about 1.5 bits per symbol more than 4 character groups.

In [8]:
def single_catalog_entropy( catalog ):
    ''' calculate the entropy in bits for an occurrence histogram
    '''
    counts = [catalog[symbol] for symbol in catalog.keys()]  
    prob = counts / np.sum(counts)
    entropy = -np.sum( prob * np.log2(prob) )
    return entropy

In [9]:
char_entropy = {}
for n in [1,2,3,4,5]:
    catalog = build_catalog_single( char_list, n)
    char_entropy[n] = single_catalog_entropy(catalog)
print('character entropy: \n', char_entropy)

print('\n incremental entropy')
for n in [2,3,4,5]:
    print(char_entropy[n] - char_entropy[n-1])

#print('\n', -1.0/27.0 * np.log2(1/27.0) * 27)

character entropy: 
 {1: 4.0952814218255424, 2: 7.458174869434918, 3: 10.098590230404971, 4: 12.11200082036454, 5: 13.678302669204667}

 incremental entropy
3.362893447609376
2.640415360970053
2.0134105899595696
1.5663018488401264


### Word sequences

The entropy per word is roughly 9.4 bits, which is only slightly less than 10.1 bits per 3 characters. Adding a second word provides about 3.6 bits, while a third word gives less than 1 bit per symbol. This means that if we already know three words in a sequence then we can predict the next word with better than 50% accuracy.

In [10]:
word_entropy = {}
for n in [1,2,3]:
    catalog = build_catalog_single( word_list, n)
    word_entropy[n] = single_catalog_entropy(catalog)
print('\nword entropy: \n', word_entropy)
#print(nentropy[1:] - nentropy[0:-1] )


word entropy: 
 {1: 9.401312892978655, 2: 13.995995066223381, 3: 14.921734766322679}


## Joint probability

In [11]:
def build_catalog_double( data ):
    ''' for each symbol (ie. character or word), 
    determine the distribution of the next symbol
    '''
    result = {}
    for indx in range(len(data)-2):
        item1, item2 = data[indx:indx+2] 
        
        # use the empty string to count level1
        if item1 not in result:
            result[item1] = {'':0}
        
        result[item1][''] += 1
        
        if item2 not in result[item1]:
            result[item1][item2] = 0
            
        result[item1][item2] += 1            
    
    return result

In [12]:
word_catalog = build_catalog_double( word_list)
word_unique = sorted( word_catalog['to'].keys() )
print([(c,word_catalog['to'][c]) for c in word_unique[0:19]])

[('', 818), ('a', 18), ('abide', 1), ('act', 2), ('all', 2), ('and', 2), ('any', 4), ('anyone', 1), ('are', 1), ('arraign', 1), ('be', 36), ('bear', 3), ('beard', 1), ('bed', 4), ('beg', 1), ('begin', 1), ('beguile', 1), ('believe', 1), ('blame', 3)]


In [13]:
def probability_single( data, catalog=None, group=1 ):
    ''' Walk through a sequence of symbols and determine the total probability
    for independent outcomes by multiplying individual probabilities
    '''
    
    if catalog is None:
        catalog = build_catalog_single(data, group = group)
    
    symbol_unique = sorted( list(catalog.keys()) )
    counts = np.array( [catalog[w] for w in symbol_unique] )
    probs = counts / np.sum(counts)
    logprobs = np.log2(probs)
    
    logprob = 0.0
    for indx in range(len(data)-group):
        symbol= ','.join(data[indx:indx+group])
        #if symbol not in symbol_unique:
        #    print('boom: ', indx, nitem, symbol, data[indx:indx+nitem])
        #    continue
        logprob += logprobs[ symbol_unique.index(symbol) ]
        
    return logprob / group

#probability_single(char_list, nitem=3) 

### independent (grouped)  probabilities

In [14]:
logprob = {}
for n in [1,2,3]:
    name = 'independent char, n={}'.format(n)
    logprob[name] = probability_single(char_list, group=n) 
    print(name, logprob[name])
    
for n in [1,2]:
    name = 'independent word, n={}'.format(n)
    logprob[name] = probability_single(word_list, group=n)     
    print(name, logprob[name])

independent char, n=1 -757917.8280183714
independent char, n=2 -690142.2115430712
independent char, n=3 -622978.6651168145
independent word, n=1 -329243.37882501545
independent word, n=2 -245069.87360965964


## Joint probability

Start with the probability of getting the first character

$$ p(c_1) $$

next, use the probability of getting the second character following the first

$$ p(c_2 | c_1 )$$

and so on

$$ p = p(c_1) \,\times\, p(c_2 | c_1) \,\times\, \ldots p(c_n, c_{-1}) $$

In [15]:
def probability_double( data, catalog=None):
    
    if catalog is None:
        catalog = build_catalog_double(data)
    
    #symbol_unique = sorted( [w for w in catalog.keys() ] )
    #counts = np.array( [catalog[w] for w in symbol_unique] )
    #probs = counts / np.sum(counts)
    #logprobs = np.log2(probs)
    
    symbol1 = data[0]
    logprob = np.log2( catalog[symbol1][''] / len(data) ).astype(np.float64)
    
    for symbol2 in data[1:]:
        cat = catalog[symbol1]
        logprob += np.log2( cat[symbol2] / cat[''] )
        symbol1 = symbol2
        
    return logprob

In [16]:
logprob['joint char'] = probability_double(char_list)
logprob['joint word'] = probability_double(word_list)
print(logprob)

{'independent char, n=1': -757917.8280183714, 'independent char, n=2': -690142.2115430712, 'independent char, n=3': -622978.6651168145, 'independent word, n=1': -329243.37882501545, 'independent word, n=2': -245069.87360965964, 'joint char': -622383.5436114544, 'joint word': -160919.94903523603}


Using joint statistics increases the likelihood of getting an outcome by more than adding a 3rd character.  Similarly, with joint word statistics we find the outcome is hugely more likely than considering pairs of words as being purely independent.

In [17]:
namelist = list(logprob.keys()) ; print(namelist)
argsort = np.argsort( [logprob[name] for name in namelist])
linelist = ['{:15.8} {}'.format(logprob[name], name) for name in namelist]
print('\n'.join( sorted( linelist ) ) )

['independent char, n=1', 'independent char, n=2', 'independent char, n=3', 'independent word, n=1', 'independent word, n=2', 'joint char', 'joint word']
     -160919.95 joint word
     -245069.87 independent word, n=2
     -329243.38 independent word, n=1
     -622383.54 joint char
     -622978.67 independent char, n=3
     -690142.21 independent char, n=2
     -757917.83 independent char, n=1


## Fake spam

We can assemble sequences of characters or words that are randomly drawn from observed distributions.  Results for independent characters are gibberish, and adding inter-character relationships doesn't help much.

Sequences of independent randomly selected words have a superficial resemblance to English, but on closer examination make no sense.

Random sequences of related words almost sound meaningful when read aloud.

In [18]:
def fakespam( data, size=99, independent=False):
    
    catalog = build_catalog_double(data)
    
    # level1 statistics
    items = [i for i in catalog.keys() ]
    counts = np.array( [catalog[i][''] for i in items] )
    probs = counts / np.sum(counts)    
    
    if independent:
        fakespam = np.random.choice(items, size=99, p=probs )
        return fakespam
    
    fakespam = []
    fakespam.append( np.random.choice(items, size=1, p=probs )[0] )  #first item
    for n in range(size):
        previous_item = catalog[fakespam[-1]]
        next_itemlist = [i for i in previous_item.keys() if i != '']
        counts = [previous_item[i] for i in next_itemlist]
        probs = np.array(counts) / np.sum(counts)
        fakespam.append( np.random.choice(next_itemlist, size=1, p=probs )[0] )
        
    return fakespam    

In [19]:
print('random characters\n', ''.join( fakespam(char_list, independent=True, size=299)), '\n' )        
print('related characters\n', ''.join( fakespam(char_list, independent=False, size=299)), '\n' )    
print('random words: \n',' '.join( fakespam(word_list, independent=True)), '\n' ) 
print('related words: \n', ' '.join( fakespam(word_list, independent=False)) )

random characters
 lh osuencuiiahsga esdbwouw sih  t  a  rs oasniotimeu  sea orr  doo  leoytgtr  eopeoeb dit is  teo   

related characters
  ndas mar u jele nd dere inu bulliteye f r andone hechefll bomewindeat tmblindooy elel amice chamy theinth plas iny hare ama aser thishedellle imso ton amll s a fores w thamy he oty monthio ply yed gioffe ss se k z dinsus mator vet myo f s s whabereas astheg icr thiki n slmyos ar dedint fui be he sr 

random words: 
 you sure and constantly you of lets up have your i the rapier heart him us that tonight element son perchance designd so crowns king poison but for again agree set him every own not feeds my my gives and have action marcellus sends hamlet but to displaying servd o walk donations i no must offence highest what his not lords but are by worm than polonius bet o note of horatio door may you absurd guilty land of know why do drooping rosencrantz beard this and is for trumpet my and my should such to that well takes 

related words: 
 you s

### Waiting for "To be or not to be"

    probability per monkey * number of monkeys * number of keys per monkey

Let us consider only the classic fragment "to be or not to be'.  The probability of getting this outcome for related key strokes is roughly 2e-19, which is much much much larger than most of the numbers which we have encountered recently.  This means that the desired output is likely to occur if 6 billion typeists were each hitting 1 key per second for 25 years.

Working with related words drastically increases the probability to 6e-10, which would be likely after only 30 minutes with a million monkeys.

In [20]:
log_change = logprob['joint word']  - logprob['independent char, n=1'] 

mass = {'population':6e9, 'earth':5.972e24, 'sun':1.989e30, 'galaxy':6e42, 'universe':6.0e51}
time = {'day':86400, 'universe':4.1e17} 

print (np.log(mass['universe']/9.11e-31) + np.log(time['universe']*1e9 ) )
logprob['joint word'] + np.log(mass['universe']/9.11e-31) + np.log(time['universe']*1e9)

249.67256377502338


-160670.276471461

In [21]:
char_logprob = probability_double('to be or not to be' , catalog=build_catalog_double(char_list)) 
word_logprob = probability_double('to be or not to be'.split() , catalog=build_catalog_double(word_list)) 

print( np.exp(char_logprob))
print( np.exp(word_logprob))

print( np.exp(char_logprob)  * mass['population'] * time['day']*365*25 )
print( np.exp(word_logprob)  * 1e6 * (60*40) )

#print( np.exp(char_logprob + np.log(mass['earth']) + np.log(time['day']*365*1e6 )))
#print( np.exp(word_logprob + np.log(mass['earth']) + np.log(time['day'] )))

2.2233646883344667e-19
6.071506486153084e-10
1.0517404321697361
1.4571615566767402


## Don't hold your breath

Imagine that the entire mass of our universe was converted to electrons and each electron could somehow hit one key per nanosecond for the entire age of the universe.  This would give a very large number of attempts

 $$\exp(250) $$
 
but this is overwhelmed by the miniscule log-probability of order -160000. 

Conclusion: Shakespeare got lucky.

## Summary

A finite number of monkeys might eventually be able to generate a small part of Hamlet's soliloquy, but producing the entire play through random chance is effectively impossible.

### Question \#1: what is the entropy of "simplified Hamlet" (Spamlet)?
    
### Question \#2: what is the probability that a monkey with a uniform random selection of 27-keys would produce Spamlet?  In other words, how many different sequences with 167774 characters are there?

### Question \#3: how does the probability change if the chance of hitting any given key was not 1/27, but the same as the distribution of Spamlet?

### Question \#4: determine the joint probability of each 2-key sequence eg. 'aa', 'ab', 'ac' from Spamlet.  How does the probability of producing Spamlet change if the monkey hits keys according to this distribution?

### Question \#5: calculate the entropy of  2-key sequences, and entire words for Spamlet.

### Optional Question \#6: write a program to generate sequences of text that sound somewhat like Shakespeare.  See for inspiration http://www.elsewhere.org/journal/pomo/ 

## Grading rubric

I generally mark assignments by starting at 10/10 and subtracting with a resolution of 1/2.  

* introduction/overview: what are we trying to do and why?
    * completely missing -1
    * inadequate/problematic -1/2 (eg. In this assignment I answered the questions)
    
* conclusions/summary: what did we do (could be in intro)?  how does it fit together?
    * completely missing -1
    * inadequate/problematic -1/2
    
* graphing problems: axes labels, titles, legends, not horribly cluttered, etc.
    * one minor -1/2
    * multiple -1
    
* interstitial text: the average page could be mostly code and output, but some bridging discussion is required
    * inadequate -1/2    
    * essentially none -1
    
* function docstring (can sometimes skip for utterly trivial function)
    * missing one -1/2
    * missing many -1
    
* duplicated code: should be factored into common functions
    * several repeated blocks of multiple lines -1/2
    * no attempt to remove many redundancies -1
    
For this assignment there should be numerical values for
    1. single symbol entropy
    2. uniform probability (1/27)^n
    3. empirical probability (1/n1)^n1 * (1/n2)^n2 ...
    4. empirical probability & entropy for pairs of characters 
    5. entropy for words
These may not be exactly the same as my results, depending on how they trimmed the text.
I also did many other calculations that aren't required.

* numerical/calculation/conceptual errors
    * one minor -1/2
    * multiple minor or one major -1

    
Either one of question 5 or 6 is acceptable.    
    
Reality check: make a note if the assignment is basically ok, but the final score is 5 or less.
Use your judgement and let me know if you are seeing common patterns. 
